In [4]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [2]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)


In [ ]:
import json
data = json.load(open("professors.json"))
data['professors']

In [12]:
index = pc.Index("rag")

In [16]:
processed_data = []
client = OpenAI()

#for review in data['professors']:
#    response = client.embeddings.create(
#        input=review["review"],
#        model="text-embedding-3-small",
#    )
#    embedding = response.data[0].embedding
#    processed_data.append({
#        "values": embedding,
#        "id": review["professor"],
#        "metadata":{
#            "review": review["review"],
#            "subject": review["subject"],
#            "stars": review["stars"]
#        }
#    })

#for professor in data['professors']:
#    for review in professor['reviews']:
#        response  = client.embeddings.create(
#        input=review["review"],
#        model="text-embedding-3-small",
#    )
#    embedding = response.data[0].embedding
#
#    metadata = {
#        "professor_name": professor['name'],
#        "professor_id": professor['id'],
#        "department": professor['department'],
#        "overall_quality": professor['overallQuality'],
#        "difficulty": professor['difficulty'],
#        "class": review['class'],
#        "review_quality": review['quality'],
#        "review_difficulty": review['difficulty']
#    }
#    processed_data.append({
#        "values": embedding,
#        "id": f"{professor['id']}-{review['class']}",
#        "metadata": metadata
#    })
#    index.upsert("rag", [(f"{professor['id']}-{review['class']}", embedding, metadata)])


for professor in data['professors']:
    reviews_text = " ".join([review['review'] for review in professor['reviews']])
    print(reviews_text)
    break
    response = client.embeddings.create(
        input=reviews_text,
        model="text-embedding-3-small",
    )
    embedding_vector = response.data[0].embedding
    index.upsert([
        {
            'id': str(professor['id']),
            'vector': embedding_vector,
            'metadata': {
                'name': professor['name'],
                'department': professor['department'],
                'overallQuality': professor['overallQuality'],
                'difficulty': professor['difficulty'],
                'reviews': professor['reviews']
            }
        }
    ])

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 20151 tokens (20151 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [8]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [9]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}